# import structure and parse

In [150]:
import urllib.request, json, numpy as np, urllib.request, json#, urllib2
with urllib.request.urlopen("https://bitbucket.org/sonnhammergrni/gs-datasets/raw/d2047430263f5ffe473525c74b4318f723c23b0e/N50/Tjarnberg-ID266081-D20151111-N50-E150-SNR10-IDY266081.json") as url:
    data = json.loads(url.read().decode())

# data = json.load(open('Nordling-ID1446937-D20150825-N10-E15-SNR3291-IDY15968.json'))
data=dict(data['obj_data'])
with urllib.request.urlopen("https://bitbucket.org/sonnhammergrni/gs-networks/raw/0b3a66e67d776eadaa5d68667ad9c8fbac12ef85/random/N50/Tjarnberg-D20150910-random-N50-L162-ID266081.json") as url:
    network = json.loads(url.read().decode())
# network = json.load(open('Nordling-D20100302-random-N10-L25-ID1446937.json'))
network=dict(network['obj_data'])
# with urllib.request.urlopen("https://bitbucket.org/sonnhammergrni/gs-datasets/src/b7e6e3dfda854fc6331099a77d2483e586a44cd8/N10/Nordling-ID1446937-D20150825-N10-E10-SNR936-IDY16058.json?at=master&fileviewer=file-view-default") as url:
#     data = json.loads(url.read().decode())
#     print(data)

Y = np.asmatrix(data['Y'])
P = np.asmatrix(data['P'])
F = np.asmatrix(data['F'])
E = np.asmatrix(data['E'])
genes = np.array(data['names'])
YY=Y-E
TA = np.asmatrix(network['A'])

# nestboot framework

In [151]:
# LS=-(P+F)*np.linalg.pinv(Y) #+E
# LS[LS<0.01]=0
# 
from sklearn.metrics import confusion_matrix, matthews_corrcoef
# y_true = np.array(TA.astype(bool).astype(int)).flatten()
# LSy_pred = np.array(LS.astype(bool).astype(int)).flatten()
# LSMCC=matthews_corrcoef(y_true, LSy_pred)
# print(LSMCC)

##NESTBOOT implementation
def LSNB(Y,R=10,Q=10,z=np.logspace(0,-6,30),r=0):
    Ge=Y.shape[0];Ex=Y.shape[1];
    LS=np.ndarray(shape=(Ge,Ge,R,Q,))#,z.shape[0]))
    sLS=np.ndarray(shape=(Ge,Ge,z.shape[0],R,Q))
    if r==1:
        B=np.random.choice(list(range(Ex)),size=Ex,replace=True)
        Y=Y[:,B]
    for r in range(1, R):
        for q in range(1, Q):
            B=np.random.choice(list(range(Ex)),size=Ex,replace=True)
            B=Y[:,B]
            while np.linalg.matrix_rank(B) < np.minimum(Ge,Ex): ##update to first
                B=np.random.choice(list(range(Ex)),size=Ex,replace=True)
                B=Y[:,B]
            LS[:,:,r,q]=-(P+F)*np.linalg.pinv(B)
            for i in range(z.shape[0]):
                sLS[:,:,i,:,:]=np.greater(LS,z[i])#LS[LS<z[i]]=0
            return np.array(sLS)

# 


# nestboot analysis & sparsity

In [152]:
z=np.logspace(-6,0,30)
R=5;Q=5;
# for i in z.shape[1]:
#     d[d<z[i]]=0
Ge=Y.shape[0];Ex=Y.shape[1];
d=LSNB(Y,R,Q,z)
B=np.random.choice(list(range(Ex)),size=Ex,replace=True)
sY=Y[:,B]
s=LSNB(sY,R,Q,z,1)

frac=np.sum(d.astype(bool),axis=4)
sfrac=np.sum(s.astype(bool),axis=4)

sign=np.sum(d>0,axis=4)
ssign=np.sum(s>0,axis=4)

/home/radonn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in greater


In [153]:
from functools import reduce
def accumulated(frac,z):
    freq=np.ndarray(shape=(frac.shape[0]*frac.shape[0],R,Q))
    x=frac.shape[0];y=frac.shape[0];z=frac.shape[2]
    F=np.vstack(frac)
    F=np.vstack(F)
    # Fint = F[:,1]&
    A=F[:,0];O=F[:,0]
    for i in range(1,R):
        AA = A&F[:,i]
        OO = O|F[:,i]
        A=AA
        O=OO


    a=np.reshape(AA,(x,y,z))
    o=np.reshape(OO,(x,y,z))
    return [a,o]


In [160]:
# sup=accumulated(sign,ssign,z)
# accumulated(frac,sfrac,z).shape
# np.divide(list(range(R)),R)
# R
# np.histogram(sign,)
# sup[1]
# np.sum(frac,axis=3)
# np.vstack(frac[:,:,r,q]).shape
# sign[:,:,2,2]
AA,OO=accumulated(frac,z)
Lsupp=AA/OO

AAA,OOO=accumulated(sign,z)
Ssupp=AAA/OOO
frac.shape

/home/radonn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()
/home/radonn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


(50, 50, 30, 5)

In [161]:
for r in range(1,R):
    for zz in range(1,z):
        freq[:,:,zz]=np.histogram(frac[:,:,zz,R],np.divide(list(range(1,R)),R)) ## this may work

IndentationError: expected an indented block (<ipython-input-161-b9e19ca3e1a1>, line 2)

# PLS, Lasso, elastic net, TLS implementations

In [4]:
from sklearn.cross_decomposition import PLSRegression
pls2 = PLSRegression(n_components=2)
pls2.fit_transform(YY,y=P)
PLS=pls2.predict(YY)
PLS[PLS<0.01]=0

In [5]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.01)
clf.fit(YY.T,P.T)
# print(A1)
L1=(clf.coef_)


In [6]:
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
# def ridge(tY,P):
regr = ElasticNet(l1_ratio=.01)
regr.fit(YY.T, P.T)
# regr.path(Y+E,P,l1_ratio=.5,alpha=0.5)
# print(a[1])
# return
# print(regr.coef_)
L0=(regr.coef_) 
# print(regr.intercept_) 
# print(regr.score(Y-E,P)) 

In [7]:
# def tls(tY,P):
Z= np.concatenate((np.asarray(YY.T),np.asarray(-P.T)),axis=1)
U,s,V = np.linalg.svd(Z)
VXY=V[0:len(Y),0+len(Y):len(Z.T)]
VYY=V[0+len(Y):len(Z.T),0+len(Y):len(Z.T)]
TLS= -(VXY/VYY)
TLS[TLS<0.01]=0
#     S[[0+len(Y):],[(0+len(Y)):]]=null
#     hatC/=V*S*V
#     hatP=hatC[:[0,0,len(Y)]]
#     hatY=hatC[:[(0+len(Y)):]]
#     return

In [8]:
from sklearn.metrics import confusion_matrix, matthews_corrcoef
y_true = np.array(TA.astype(bool).astype(int)).flatten()
LSy_pred = np.array(LS.astype(bool).astype(int)).flatten()
PLSy_pred = np.array(PLS.astype(bool).astype(int)).flatten()
L1y_pred = np.array(L1.astype(bool).astype(int)).flatten()
L0y_pred = np.array(L0.astype(bool).astype(int)).flatten()
TLSy_pred = np.array(TLS.astype(bool).astype(int)).flatten()

# cm=confusion_matrix(y_true, y_pred)
# TP = cm[0][0]
# FP = cm[0][1]
# FN = cm[1][0]
# TN = cm[1][1]

LSMCC=matthews_corrcoef(y_true, LSy_pred)
L1MCC=matthews_corrcoef(y_true, L1y_pred)
L0MCC=matthews_corrcoef(y_true, L0y_pred)
TLSMCC=matthews_corrcoef(y_true, TLSy_pred)
# PLSMCC=matthews_corrcoef(y_true, PLSy_pred)

print(LSMCC,L1MCC,L0MCC,TLSMCC)#,PLSMCC)

0.57735026919 0.866025403784 0.40618119723 -0.0230940107676


In [9]:
from cvxpy import *
y_true

array([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1])

In [22]:
#https://github.com/cmap/cmapPy/blob/master/tutorials/cmapPy_pandasGEXpress_tutorial.ipynb
import cmapPy
run_filter_query(resource_name, filter_clause)


NameError: name 'run_filter_query' is not defined

In [17]:
import os
# import grep
os.getcwd()
# os.chdir(’/home’) 
# os.listdir(".")
grep.grep("*.py") 
# models/samples/core/get_started/premade_estimator

NameError: name 'grep' is not defined